In [1]:
import importlib
import json

from amlta.question_generation import generate, process

generate = importlib.reload(generate)
process = importlib.reload(process)

In [5]:
r = generate.generate_random_process_question()
r_parsed = r.choices[0].message.parsed
if r_parsed:
    print(r_parsed.model_dump_json(indent=2))

<process>
Name: Wärme-Heizen-mix-VN-HH/KV-2015

Year:
- '2015'

Geography: VN

Class:
- Verbrauch-Haushalte

Technology:
- 'Verteilung der Endenergienachfrage für Raumwärme im Bereich Haushalten und Kleinverbrauch auf Energieträger; Daten nach #1'

Main Output:
- 1 TJ  Raumwärme
- Main Output Flow: Flow("space heat" output, 1e+06 MJ (Net calorific value), type='Product flow', class='Systems / Other systems')
</process>

<query_params>
query_type: type
direction: both
aggregation: list
flow_type: Waste flow
</query_params>

Given the provided LCI process data and query context, what is a question an analyst could ask
about the process flows that match the query params?

Let's think step by step.
{
  "flow_question": {
    "thoughts": [
      "The process involves heat distribution in households and small consumers in Vietnam in 2015.",
      "The query asks for flows with a flow type of 'Waste flow', and it should include both inputs and outputs, aggregated in a list.",
      "In the co

In [24]:
batch_in_path = generate.prepare_batch(
    n=200,
    offset=800,
    questions_per_process=3,
    model="gpt-4o-mini",
)

In [25]:
batch = generate.send_batch(batch_in_path)

In [26]:
batch

Batch(id='batch_67b22982fbdc8190999b0dd1b1b2437a', completion_window='24h', created_at=1739729283, endpoint='/v1/chat/completions', input_file_id='file-MNNXWVJmQ9gktaCKb2RWjB', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1739815683, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [29]:
batch_out_path = generate.retrieve_batch_results(
    "batch_67b22982fbdc8190999b0dd1b1b2437a"
)

Batch(id='batch_67b22982fbdc8190999b0dd1b1b2437a', completion_window='24h', created_at=1739729283, endpoint='/v1/chat/completions', input_file_id='file-MNNXWVJmQ9gktaCKb2RWjB', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1739730683, error_file_id=None, errors=None, expired_at=None, expires_at=1739815683, failed_at=None, finalizing_at=1739730366, in_progress_at=1739729284, metadata=None, output_file_id='file-F5oNU1rntGURC1KNkqtiB9', request_counts=BatchRequestCounts(completed=600, failed=0, total=600))


In [30]:
out_data = process.process_and_save_batch_question_results(batch_out_path)

Saved 600 questions to D:\dev\amlta-project\data\generated\questions\out\batch_gpt-4o-mini_800_1000.jsonl


In [32]:
out_data[10:20]

[{'batch': 'batch_gpt-4o-mini_800_1000',
  'process_uuid': 'd3bf4a54-2578-46e2-b7ba-ce22bf4962af',
  'question_id': 1,
  'basic_query': 'biofuel synthesis technology',
  'general_query': 'biomass to liquid fuel synthesis',
  'specific_query': 'biofuel production Fischer-Tropsch in Germany 2030',
  'flow_query_params': {'query_type': 'name',
   'direction': 'both',
   'aggregation': 'sum',
   'flow_name': 'HFC-125'},
  'question': 'What is the total amount of HFC-125 produced or consumed by <the process>?',
  'question_replaced_basic': 'What is the total amount of HFC-125 produced or consumed by biofuel synthesis technology?',
  'question_replaced_general': 'What is the total amount of HFC-125 produced or consumed by biomass to liquid fuel synthesis?',
  'question_replaced_specific': 'What is the total amount of HFC-125 produced or consumed by biofuel production Fischer-Tropsch in Germany 2030?'},
 {'batch': 'batch_gpt-4o-mini_800_1000',
  'process_uuid': 'd3bf4a54-2578-46e2-b7ba-ce22bf

In [27]:
qa_gen_out_dir = generate.get_generated_questions_path() / "out"

In [ ]:
batch_out_1 = next(qa_gen_out_dir.iterdir())
results = [json.loads(line) for line in batch_out_1.read_text().splitlines()]

In [ ]:
import pandas as pd

pd.concat(
    [process.build_training_batched_dfs(result) for result in results],
    ignore_index=True,
    axis=0,
).reset_index(drop=True)

,batch,question_id,question_template,question_basic,question_general,question_specific,process_uuid,flows_start,flows_stop,aggregation,coordinates,answers
0,batch_gpt-4o-mini_0_200,0,What are the total emissions to air from <the ...,What are the total emissions to air from energ...,What are the total emissions to air from solar...,What are the total emissions to air from elect...,404ee608-dcb1-4c7e-9bda-b5eafe763547,0,20,list,[],[]
1,batch_gpt-4o-mini_0_200,0,What are the total emissions to air from <the ...,What are the total emissions to air from energ...,What are the total emissions to air from solar...,What are the total emissions to air from elect...,404ee608-dcb1-4c7e-9bda-b5eafe763547,20,40,list,"[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1...","[3.88, 4.3, 0.0, 0.0, 0.0, 0.0, 0.008, 0.0, 0...."
2,batch_gpt-4o-mini_0_200,0,What are the total emissions to air from <the ...,What are the total emissions to air from energ...,What are the total emissions to air from solar...,What are the total emissions to air from elect...,404ee608-dcb1-4c7e-9bda-b5eafe763547,40,60,list,"[(0, 1), (1, 1), (3, 1), (4, 1), (5, 1), (6, 1...","[0.002, 0.178, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
3,batch_gpt-4o-mini_0_200,0,What are the total emissions to air from <the ...,What are the total emissions to air from energ...,What are the total emissions to air from solar...,What are the total emissions to air from elect...,404ee608-dcb1-4c7e-9bda-b5eafe763547,60,70,list,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[0.0, 0.0, 0.0, 7598.0, 23.9, 8.5, 0.0, 0.0, 0..."
4,batch_gpt-4o-mini_0_200,1,What are the input amounts of elementary flows...,What are the input amounts of elementary flows...,What are the input amounts of elementary flows...,What are the input amounts of elementary flows...,404ee608-dcb1-4c7e-9bda-b5eafe763547,0,20,list,"[(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11,...","[1190.0, 1040.0, 1210.445, 25900.0, 1000000.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3061,batch_gpt-4o-mini_0_200,2,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,71f0bce6-9190-4137-86f6-67e9473fb10d,0,20,count,[],[]
3062,batch_gpt-4o-mini_0_200,2,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,71f0bce6-9190-4137-86f6-67e9473fb10d,20,40,count,"[(3, 1), (5, 1)]","[229.0, 9.75]"
3063,batch_gpt-4o-mini_0_200,2,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,71f0bce6-9190-4137-86f6-67e9473fb10d,40,60,count,[],[]
3064,batch_gpt-4o-mini_0_200,2,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,71f0bce6-9190-4137-86f6-67e9473fb10d,60,80,count,[],[]
